# Spectral Landscape Explorer

## Philosophy: Dezoom & Discover

Instead of tunnel-visioning on a specific target value, we **explore the full landscape** of spectral properties across:

- **Metric parameters**: ratio, anisotropy, kernel bandwidth
- **Topological parameters**: H*, b₂/b₃ split
- **Construction variants**: TCS weights, distance functions

**Goal**: Find patterns, clusters, phase transitions — let the data speak.

---

### Approach

1. **Fast probes**: Small N (5k-10k) for speed, many configurations
2. **Wide grid**: Explore parameter space broadly
3. **Visualization**: 2D/3D landscapes, heatmaps
4. **ML patterns**: Regression, clustering, anomaly detection

In [ ]:
# =============================================================================
# SETUP
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from itertools import product
import warnings
import time
import json
from datetime import datetime
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')

# Style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

print(f"Landscape Explorer initialized - {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# =============================================================================
# CORE FUNCTIONS (lightweight for fast exploration)
# =============================================================================

def sample_S3(n, rng):
    """Uniform sampling on S³."""
    q = rng.standard_normal((n, 4)).astype(np.float32)
    return q / np.linalg.norm(q, axis=1, keepdims=True)

def geodesic_S3(Q1, Q2):
    """Geodesic distance on S³."""
    dot = np.clip(np.abs(Q1 @ Q2.T), 0, 1)
    return 2 * np.arccos(dot)

def geodesic_S1(t1, t2):
    """Geodesic distance on S¹."""
    diff = np.abs(t1[:, None] - t2[None, :])
    return np.minimum(diff, 2*np.pi - diff)

def compute_lambda1(D, k, sigma_factor=1.0):
    """Fast λ₁ computation from distance matrix."""
    n = D.shape[0]
    k = min(k, n-1)
    
    # Adaptive sigma
    knn = np.partition(D, k, axis=1)[:, 1:k+1]
    sigma = np.median(knn) * sigma_factor
    sigma = max(sigma, 1e-6)
    
    # Gaussian weights
    W = np.exp(-D**2 / (2*sigma**2))
    np.fill_diagonal(W, 0)
    
    # Sparsify
    for i in range(n):
        thresh = np.partition(W[i], -k)[-k]
        W[i, W[i] < thresh] = 0
    
    W = (W + W.T) / 2
    
    # Normalized Laplacian
    d = np.maximum(W.sum(1), 1e-10)
    d_inv = 1/np.sqrt(d)
    L = np.eye(n) - (d_inv[:, None] * W * d_inv[None, :])
    
    try:
        eigs, _ = eigsh(sp.csr_matrix(L), k=5, which='SM', tol=1e-6)
        eigs = np.sort(np.real(eigs))
        for e in eigs:
            if e > 1e-7:
                return float(e)
        return float(eigs[1])
    except:
        return np.nan

print("✓ Core functions loaded")

In [ ]:
# =============================================================================
# CONFIGURABLE TCS METRIC
# =============================================================================

def tcs_distance_matrix(
    N: int,
    seed: int,
    # Metric parameters (these are our exploration axes)
    ratio: float = 1.0,        # Anisotropy S³₂ vs S³₁
    alpha_s1: float = 1.0,     # S¹ weight
    det_g: float = 65/32,      # G₂ determinant
    mix_mode: str = 'standard' # 'standard', 'equal', 's3_only', 'product'
) -> np.ndarray:
    """
    Compute TCS distance matrix with configurable metric.
    
    mix_mode options:
    - 'standard': d² = α·d_S1² + d_S3₁² + r²·d_S3₂² (original TCS)
    - 'equal': d² = d_S1² + d_S3₁² + d_S3₂² (democratic)
    - 's3_only': d² = d_S3₁² + d_S3₂² (ignore S¹)
    - 'product': d = d_S1 × d_S3₁ × d_S3₂ (multiplicative)
    """
    rng = np.random.default_rng(seed)
    
    # Sample components
    theta = rng.uniform(0, 2*np.pi, N).astype(np.float32)
    q1 = sample_S3(N, rng)
    q2 = sample_S3(N, rng)
    
    # Distances on each factor
    d_s1 = geodesic_S1(theta, theta)
    d_s3_1 = geodesic_S3(q1, q1)
    d_s3_2 = geodesic_S3(q2, q2)
    
    # Combine according to mode
    if mix_mode == 'standard':
        alpha = det_g / (ratio**3) * alpha_s1
        d_sq = alpha * d_s1**2 + d_s3_1**2 + (ratio**2) * d_s3_2**2
        return np.sqrt(np.maximum(d_sq, 0))
    
    elif mix_mode == 'equal':
        d_sq = d_s1**2 + d_s3_1**2 + d_s3_2**2
        return np.sqrt(d_sq)
    
    elif mix_mode == 's3_only':
        d_sq = d_s3_1**2 + (ratio**2) * d_s3_2**2
        return np.sqrt(d_sq)
    
    elif mix_mode == 'product':
        return (d_s1 + 0.1) * (d_s3_1 + 0.1) * (d_s3_2 + 0.1)
    
    else:
        raise ValueError(f"Unknown mix_mode: {mix_mode}")

print("✓ Configurable TCS metric loaded")

In [ ]:
# =============================================================================
# PROBE LAUNCHER
# =============================================================================

def launch_probe(config: dict) -> dict:
    """
    Launch a single probe with given configuration.
    Returns results dictionary.
    """
    t0 = time.time()
    
    # Extract params
    N = config.get('N', 5000)
    k = config.get('k', 50)
    seed = config.get('seed', 42)
    H_star = config.get('H_star', 99)
    ratio = config.get('ratio', H_star/84)
    alpha_s1 = config.get('alpha_s1', 1.0)
    det_g = config.get('det_g', 65/32)
    mix_mode = config.get('mix_mode', 'standard')
    sigma_factor = config.get('sigma_factor', 1.0)
    
    # Compute
    D = tcs_distance_matrix(N, seed, ratio, alpha_s1, det_g, mix_mode)
    lambda1 = compute_lambda1(D, k, sigma_factor)
    product = lambda1 * H_star if not np.isnan(lambda1) else np.nan
    
    elapsed = time.time() - t0
    
    return {
        **config,
        'lambda1': lambda1,
        'product': product,
        'elapsed': elapsed
    }

def launch_exploration(configs: list, desc="Exploring") -> pd.DataFrame:
    """Launch multiple probes and return DataFrame."""
    results = []
    for cfg in tqdm(configs, desc=desc):
        results.append(launch_probe(cfg))
    return pd.DataFrame(results)

print("✓ Probe launcher ready")

---

## Exploration 1: Ratio Landscape

How does λ₁×H* vary with the anisotropy ratio?

In [ ]:
# Exploration 1: Ratio sweep

ratios = np.linspace(0.5, 3.0, 25)
configs_ratio = [
    {'N': 5000, 'k': 50, 'seed': s, 'H_star': 99, 'ratio': r, 'mix_mode': 'standard'}
    for r in ratios
    for s in [42, 123]
]

print(f"Launching {len(configs_ratio)} probes for ratio exploration...")
df_ratio = launch_exploration(configs_ratio, "Ratio sweep")

In [ ]:
# Plot ratio landscape

fig, ax = plt.subplots(figsize=(10, 6))

df_agg = df_ratio.groupby('ratio').agg({'product': ['mean', 'std']}).reset_index()
df_agg.columns = ['ratio', 'mean', 'std']

ax.fill_between(df_agg['ratio'], df_agg['mean']-df_agg['std'], 
                df_agg['mean']+df_agg['std'], alpha=0.3)
ax.plot(df_agg['ratio'], df_agg['mean'], 'o-', markersize=6, linewidth=2)

# Reference lines
ax.axhline(y=13, color='red', linestyle='--', label='Target = 13')
ax.axvline(x=99/84, color='green', linestyle=':', alpha=0.7, label=f'H*/84 = {99/84:.2f}')

ax.set_xlabel('Ratio (anisotropy parameter)', fontsize=12)
ax.set_ylabel('λ₁ × H*', fontsize=12)
ax.set_title('Spectral Landscape: λ₁×H* vs Anisotropy Ratio', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('landscape_ratio.png', dpi=150)
plt.show()

# Find where it crosses 13
cross_idx = np.where(np.diff(np.sign(df_agg['mean'] - 13)))[0]
if len(cross_idx) > 0:
    cross_ratio = df_agg['ratio'].iloc[cross_idx[0]]
    print(f"\n★ Crosses 13 at ratio ≈ {cross_ratio:.2f}")

---

## Exploration 2: H* Landscape

How does λ₁×H* behave for different topologies (different H* values)?

In [ ]:
# Exploration 2: H* sweep

H_values = [20, 40, 60, 80, 99, 120, 150, 200, 300]

configs_H = [
    {'N': 5000, 'k': 50, 'seed': s, 'H_star': H, 'ratio': H/84, 'mix_mode': 'standard'}
    for H in H_values
    for s in [42, 123, 456]
]

print(f"Launching {len(configs_H)} probes for H* exploration...")
df_H = launch_exploration(configs_H, "H* sweep")

In [ ]:
# Plot H* landscape

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: λ₁×H* vs H*
ax1 = axes[0]
df_H_agg = df_H.groupby('H_star').agg({'product': ['mean', 'std'], 'lambda1': 'mean'}).reset_index()
df_H_agg.columns = ['H_star', 'product_mean', 'product_std', 'lambda1_mean']

ax1.errorbar(df_H_agg['H_star'], df_H_agg['product_mean'], 
             yerr=df_H_agg['product_std'], fmt='o-', capsize=5, markersize=8)
ax1.axhline(y=13, color='red', linestyle='--', label='Target = 13')
ax1.set_xlabel('H* (topology parameter)', fontsize=12)
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Spectral Product vs Topology', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: λ₁ vs H* (should scale as 1/H*)
ax2 = axes[1]
ax2.loglog(df_H_agg['H_star'], df_H_agg['lambda1_mean'], 'o-', markersize=8)

# Fit power law
log_H = np.log(df_H_agg['H_star'])
log_lam = np.log(df_H_agg['lambda1_mean'])
slope, intercept = np.polyfit(log_H, log_lam, 1)
H_fit = np.linspace(df_H_agg['H_star'].min(), df_H_agg['H_star'].max(), 100)
lam_fit = np.exp(intercept) * H_fit**slope
ax2.plot(H_fit, lam_fit, 'r--', label=f'Fit: λ₁ ∝ H*^{slope:.2f}')

ax2.set_xlabel('H*', fontsize=12)
ax2.set_ylabel('λ₁', fontsize=12)
ax2.set_title(f'Eigenvalue Scaling (slope = {slope:.2f})', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('landscape_Hstar.png', dpi=150)
plt.show()

print(f"\n★ Power law: λ₁ ∝ H*^{slope:.3f}")
print(f"  If slope ≈ -1, then λ₁×H* = const (universal law)")

---

## Exploration 3: Metric Mode Comparison

How do different metric constructions affect the spectral product?

In [ ]:
# Exploration 3: Mix mode comparison

modes = ['standard', 'equal', 's3_only']

configs_mode = [
    {'N': 5000, 'k': 50, 'seed': s, 'H_star': 99, 'ratio': 99/84, 'mix_mode': m}
    for m in modes
    for s in [42, 123, 456, 789]
]

print(f"Launching {len(configs_mode)} probes for mode comparison...")
df_mode = launch_exploration(configs_mode, "Mode comparison")

In [ ]:
# Plot mode comparison

fig, ax = plt.subplots(figsize=(8, 6))

df_mode_agg = df_mode.groupby('mix_mode')['product'].agg(['mean', 'std']).reset_index()

bars = ax.bar(df_mode_agg['mix_mode'], df_mode_agg['mean'], 
              yerr=df_mode_agg['std'], capsize=8, color=['#2E86AB', '#A23B72', '#F18F01'])

ax.axhline(y=13, color='red', linestyle='--', linewidth=2, label='Target = 13')

ax.set_xlabel('Metric Construction', fontsize=12)
ax.set_ylabel('λ₁ × H*', fontsize=12)
ax.set_title('Impact of Metric Construction on Spectral Product', fontsize=14)
ax.legend()

# Annotate values
for bar, row in zip(bars, df_mode_agg.itertuples()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            f'{row.mean:.1f}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig('landscape_modes.png', dpi=150)
plt.show()

---

## Exploration 4: 2D Parameter Space

Heatmap of λ₁×H* across (ratio, k) space

In [ ]:
# Exploration 4: 2D heatmap

ratios_2d = np.linspace(0.5, 2.5, 15)
ks_2d = [20, 30, 40, 50, 60, 80, 100]

configs_2d = [
    {'N': 5000, 'k': k, 'seed': 42, 'H_star': 99, 'ratio': r, 'mix_mode': 'standard'}
    for r in ratios_2d
    for k in ks_2d
]

print(f"Launching {len(configs_2d)} probes for 2D heatmap...")
df_2d = launch_exploration(configs_2d, "2D grid")

In [ ]:
# Plot 2D heatmap

pivot = df_2d.pivot(index='k', columns='ratio', values='product')

fig, ax = plt.subplots(figsize=(12, 6))

im = ax.imshow(pivot.values, aspect='auto', cmap='RdYlGn_r',
               extent=[ratios_2d.min(), ratios_2d.max(), ks_2d[-1], ks_2d[0]],
               vmin=10, vmax=20)

# Contour at 13
X, Y = np.meshgrid(ratios_2d, ks_2d)
cs = ax.contour(X, Y, pivot.values, levels=[13], colors='black', linewidths=2)
ax.clabel(cs, inline=True, fontsize=10, fmt='%.0f')

plt.colorbar(im, ax=ax, label='λ₁ × H*')
ax.set_xlabel('Ratio (anisotropy)', fontsize=12)
ax.set_ylabel('k (neighbors)', fontsize=12)
ax.set_title('Spectral Landscape: (ratio, k) → λ₁×H*', fontsize=14)

plt.tight_layout()
plt.savefig('landscape_2d_heatmap.png', dpi=150)
plt.show()

---

## Exploration 5: ML Pattern Discovery

Can we find a predictive formula for λ₁×H*?

In [ ]:
# Large-scale exploration for ML

np.random.seed(42)

# Random sampling of parameter space
n_samples = 200

configs_ml = []
for i in range(n_samples):
    configs_ml.append({
        'N': 5000,
        'k': int(np.random.uniform(20, 100)),
        'seed': np.random.randint(1, 10000),
        'H_star': int(np.random.uniform(30, 200)),
        'ratio': np.random.uniform(0.5, 2.5),
        'alpha_s1': np.random.uniform(0.5, 2.0),
        'sigma_factor': np.random.uniform(0.5, 2.0),
        'mix_mode': 'standard'
    })

print(f"Launching {len(configs_ml)} random probes for ML...")
df_ml = launch_exploration(configs_ml, "ML sampling")

# Clean NaN
df_ml = df_ml.dropna(subset=['product'])
print(f"Valid samples: {len(df_ml)}")

In [ ]:
# Train ML model to predict λ₁×H*

features = ['k', 'H_star', 'ratio', 'alpha_s1', 'sigma_factor']
X = df_ml[features].values
y = df_ml['product'].values

# Polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X, y)

# Feature importance
importance = pd.DataFrame({
    'feature': features,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\n★ Feature Importance (Random Forest):")
print(importance.to_string(index=False))

# R² score
r2 = rf.score(X, y)
print(f"\nR² = {r2:.3f}")

In [ ]:
# Visualize feature importance

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Feature importance
ax1 = axes[0]
ax1.barh(importance['feature'], importance['importance'], color='steelblue')
ax1.set_xlabel('Importance', fontsize=12)
ax1.set_title('What Drives λ₁×H*?', fontsize=14)
ax1.invert_yaxis()

# Right: Predicted vs Actual
ax2 = axes[1]
y_pred = rf.predict(X)
ax2.scatter(y, y_pred, alpha=0.5, s=30)
ax2.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2)
ax2.set_xlabel('Actual λ₁×H*', fontsize=12)
ax2.set_ylabel('Predicted λ₁×H*', fontsize=12)
ax2.set_title(f'ML Prediction (R² = {r2:.3f})', fontsize=14)

plt.tight_layout()
plt.savefig('landscape_ml.png', dpi=150)
plt.show()

---

## Exploration 6: Clustering Analysis

Are there distinct "phases" in the spectral landscape?

In [ ]:
# Clustering

# Prepare features
X_cluster = df_ml[['ratio', 'H_star', 'k', 'product']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_ml['cluster'] = kmeans.fit_predict(X_scaled)

# t-SNE for visualization
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: t-SNE colored by cluster
ax1 = axes[0]
scatter1 = ax1.scatter(X_tsne[:, 0], X_tsne[:, 1], c=df_ml['cluster'], 
                       cmap='tab10', s=50, alpha=0.7)
ax1.set_xlabel('t-SNE 1')
ax1.set_ylabel('t-SNE 2')
ax1.set_title('Spectral Landscape Clusters', fontsize=14)
plt.colorbar(scatter1, ax=ax1, label='Cluster')

# Right: t-SNE colored by λ₁×H*
ax2 = axes[1]
scatter2 = ax2.scatter(X_tsne[:, 0], X_tsne[:, 1], c=df_ml['product'], 
                       cmap='viridis', s=50, alpha=0.7)
ax2.set_xlabel('t-SNE 1')
ax2.set_ylabel('t-SNE 2')
ax2.set_title('Colored by λ₁×H*', fontsize=14)
plt.colorbar(scatter2, ax=ax2, label='λ₁×H*')

plt.tight_layout()
plt.savefig('landscape_clusters.png', dpi=150)
plt.show()

# Cluster statistics
print("\n★ Cluster Statistics:")
print(df_ml.groupby('cluster')['product'].agg(['mean', 'std', 'count']))

---

## Summary & Export

In [ ]:
# Compile all results

summary = {
    'timestamp': datetime.now().isoformat(),
    'explorations': {
        'ratio_sweep': len(df_ratio),
        'H_sweep': len(df_H),
        'mode_comparison': len(df_mode),
        '2d_grid': len(df_2d),
        'ml_sampling': len(df_ml)
    },
    'key_findings': {
        'eigenvalue_scaling': f'λ₁ ∝ H*^{slope:.3f}',
        'ml_r2': float(r2),
        'top_feature': importance.iloc[0]['feature'],
        'n_clusters': 4
    }
}

# Save
with open('landscape_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

# Save full ML dataset
df_ml.to_csv('landscape_ml_data.csv', index=False)

print("\n" + "="*60)
print("LANDSCAPE EXPLORATION COMPLETE")
print("="*60)
print(f"\nTotal probes: {len(df_ratio) + len(df_H) + len(df_mode) + len(df_2d) + len(df_ml)}")
print(f"\nKey findings:")
print(f"  • Eigenvalue scaling: λ₁ ∝ H*^{slope:.3f}")
print(f"  • ML predictability: R² = {r2:.3f}")
print(f"  • Most important feature: {importance.iloc[0]['feature']}")
print(f"\nFiles saved:")
print(f"  • landscape_*.png (visualizations)")
print(f"  • landscape_summary.json")
print(f"  • landscape_ml_data.csv")